In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
     housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(
     X_train_full, y_train_full)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [2]:
from tensorflow import keras
'''model = keras.models.Sequential([
     keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
     keras.layers.Dense(1)
     ])
model.compile(loss="mean_squared_error", optimizer="sgd")
history = model.fit(X_train, y_train, epochs=20,
     validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)'''

'model = keras.models.Sequential([\n     keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),\n     keras.layers.Dense(1)\n     ])\nmodel.compile(loss="mean_squared_error", optimizer="sgd")\nhistory = model.fit(X_train, y_train, epochs=20,\n     validation_data=(X_valid, y_valid))\nmse_test = model.evaluate(X_test, y_test)'

In [3]:
'''X_new = X_test[:3] # pretend these are new instances
y_pred = model.predict(X_new)
y_pred'''

'X_new = X_test[:3] # pretend these are new instances\ny_pred = model.predict(X_new)\ny_pred'

In [4]:
#Functional API starts here
#Thanks to functional API we can build our own architecture for the Neural Network
'''input = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.models.Model(inputs=[input], outputs=[output])'''

'input = keras.layers.Input(shape=X_train.shape[1:])\nhidden1 = keras.layers.Dense(30, activation="relu")(input)\nhidden2 = keras.layers.Dense(30, activation="relu")(hidden1)\nconcat = keras.layers.Concatenate()([input, hidden2])\noutput = keras.layers.Dense(1)(concat)\nmodel = keras.models.Model(inputs=[input], outputs=[output])'

In [5]:
'''model.compile(loss="mean_squared_error", optimizer="adam")
history = model.fit(X_train, y_train, epochs=20,
     validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)'''

'model.compile(loss="mean_squared_error", optimizer="adam")\nhistory = model.fit(X_train, y_train, epochs=20,\n     validation_data=(X_valid, y_valid))\nmse_test = model.evaluate(X_test, y_test)'

In [6]:
#wide & deep algorithm
input_A = keras.layers.Input(shape=[5])
input_B = keras.layers.Input(shape=[6])
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.models.Model(inputs=[input_A, input_B], outputs=[output])

In [7]:
model.compile(loss="mse", optimizer="adam")
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]
history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
     validation_data=((X_valid_A, X_valid_B), y_valid))
mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_new_A, X_new_B))

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 9188.4150 - val_loss: 19.2259
Epoch 2/20
363/363 [==============================] - 1s 1ms/step - loss: 26.5629 - val_loss: 17.1965
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 23.2706 - val_loss: 15.5938
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 19.6452 - val_loss: 18.9242
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 15.6750 - val_loss: 11.8198
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 13.6745 - val_loss: 9.7404
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 25.1993 - val_loss: 8.0285
Epoch 8/20
363/363 [==============================] - 0s 1ms/step - loss: 8.9220 - val_loss: 6.8219
Epoch 9/20
363/363 [==============================] - 0s 1ms/step - loss: 6.2864 - val_loss: 4.4134
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 4.4206 - va

In [8]:
#Another way to implement wide & deep (subclassed model)
#this is way more dynamic, because we can add loops and other things inside the call() method
# the downside of this approach is that we can't save our model

class WideAndDeepModel(keras.models.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs) # handles standard args (e.g., name)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output
model2 = WideAndDeepModel()

In [9]:
# model.save("my_keras_model.h5")     use this to save the model in the folder
# model = keras.models.load_model("my_keras_model.h5")     use this in another project in the same folder to use the model